In [1]:
!pip install transformers gradio
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import os
from huggingface_hub import login

In [3]:
# 替換為你的Hugging Face API Token
my_read_token = "請換成你自己的read token"
os.environ["HUGGINGFACE_TOKEN"] = my_read_token
# 登錄Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
model_name = "taide/TAIDE-LX-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# 加载 tokenizer 和 模型

peft_name = "mark1098/TAIDE-LX-7B-Chat-Medical-Fintune"
model = PeftModel.from_pretrained(model, peft_name)
model = model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/813k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

In [5]:
import gradio as gr
import re
# 定义一个生成文本的函数
def chat_with_model(input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    # outputs = model.generate(inputs.input_ids, max_new_tokens=150, pad_token_id=tokenizer.pad_token_id)
    # prompt = "每天肚子痛是啥狀況？"
    messages = [
        {"role": "system", "content": "你是一位專業的醫療人員，請用心且專業的以三到五句話回答問題。"},
        {"role": "user", "content": input_text}
    ]

    # 將消息合併成一個文本輸入
    def format_messages(messages):
        formatted_messages = ""
        for message in messages:
            if message['role'] == 'system':
                formatted_messages += f"[SYSTEM] {message['content']}\n"
            elif message['role'] == 'user':
                formatted_messages += f"[USER] {message['content']}\n"
        return formatted_messages

    # 格式化消息
    formatted_text = format_messages(messages)
    print("Formatted text:", formatted_text)

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    print("Generated text with chat template:", text)


    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

    # 印出model_inputs進行檢查
    print("Model inputs:", model_inputs)

    # 產生文本
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=90,
        #eos_token_id=tokenizer.encode('<|eot_id|>')[0],
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=model_inputs.attention_mask,
    repetition_penalty=1.6,  # 增加 repetition_penalty
        top_k=50,  # 設定最高k個概率詞
        # stop_token=tokenizer.eos_token,  # 確保模型在遇到停止標記時中斷
        do_sample=True,
        top_p=0.15,  # 新增 top_p 參數
        temperature=0.15,  # 設定 temperature 參數
        #forced_eos_token_id=tokenizer.encode('</s>')[0]
    )
    generated_ids = generated_ids[:, model_inputs.input_ids.shape[-1]:]


    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # 清理生成的文本

    response = re.sub(r'</s>.*', '', response, flags=re.DOTALL)
    response = re.sub(r'</s>.*', '</s>', response)
    response = re.sub(r'\[.*?\]', '', response)  # 去除方括號內的內容
    response = re.sub(r'</?[^>]+>', '', response)  # 去除HTML標籤
    response = re.sub(r'dress|dressing', '', response, flags=re.IGNORECASE)  # 去除 "dress" 和 "dressing"
    response = re.sub(r'<<.*?>>', '', response)  # 去除 `<<SYS>>` 標籤
    response = re.sub(r'\[.*?\]', '', response)  # 去除 `[/EMBB]]` 標籤
    response = response.strip()  # 去除前後多餘的空格
    # 從最後開始查找句號並去除句號之後的語句
    def remove_after_last_period(text):
        last_period_index = max(text.rfind('。'), text.rfind('!'))
        if last_period_index != -1:
            return text[:last_period_index + 1]
        return text

    # 清理生成的文本
    response = remove_after_last_period(response).strip()
    return response

# 创建 Gradio 界面
iface = gr.Interface(fn=chat_with_model, inputs="text", outputs="text", title="Medical Chatbot", description="使用TAIDE-LX-7B-Chat-Medical-Fintune模型的医疗聊天机器人")

# 启动接口
iface.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5126b6b8605d5f7d39.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
